## Import

In [2]:
from datetime import datetime
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

SEED =37

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(SEED) # Seed 고정
def norm(df):
    df_normalized = (df - df.mean()) / df.std()
    df_normalized.head()
    df_normalized.dropna(axis=1)
    return df_normalized
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Data Load

In [4]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [4]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

## Data Pre-processing

In [5]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [6]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [7]:
train_x = norm(train_x)
test_x = norm(test_x)

In [8]:
len(train_x.columns),len(test_x.columns)

(2877, 2877)

In [9]:
print(train_y.unique())
train = pd.concat([train_x,train_y],axis=1)
train.head()

[1 2 0]


,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56,X_57,X_58,X_59,X_60,X_61,X_62,X_63,X_64,X_65,X_66,X_67,X_68,X_69,X_70,X_71,X_72,X_73,X_74,X_75,X_76,X_77,X_78,X_79,X_80,X_81,X_82,X_83,X_84,X_85,X_86,X_87,X_88,X_89,X_90,X_91,X_92,X_93,X_94,X_95,X_96,X_97,X_98,X_99,X_100,X_101,X_102,X_103,X_104,X_105,X_106,X_107,X_108,X_109,X_110,X_111,X_112,X_113,X_114,X_115,X_116,X_117,X_118,X_119,X_120,X_121,X_122,X_123,X_124,X_125,X_126,X_127,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,X_141,X_142,X_143,X_144,X_145,X_146,X_147,X_148,X_149,X_150,X_151,X_152,X_153,X_154,X_155,X_156,X_157,X_158,X_159,X_160,X_161,X_162,X_163,X_164,X_165,X_166,X_167,X_168,X_169,X_170,X_171,X_172,X_173,X_174,X_175,X_176,X_177,X_178,X_179,X_180,X_181,X_182,X_183,X_184,X_185,X_186,X_187,X_188,X_189,X_190,X_191,X_192,X_193,X_194,X_195,X_196,X_197,X_198,X_199,X_200,X_201,X_202,X_203,X_204,X_205,X_206,X_207,X_208,X_209,X_210,X_211,X_212,X_213,X_214,X_215,X_216,X_217,X_218,X_219,X_220,X_221,X_222,X_223,X_224,X_225,X_226,X_227,X_228,X_229,X_230,X_231,X_232,X_233,X_234,X_235,X_236,X_237,X_238,X_239,X_240,X_241,X_242,X_243,X_244,X_245,X_246,X_247,X_248,...,X_2627,X_2628,X_2629,X_2630,X_2631,X_2632,X_2633,X_2634,X_2635,X_2636,X_2637,X_2638,X_2639,X_2640,X_2641,X_2642,X_2643,X_2644,X_2645,X_2646,X_2647,X_2648,X_2649,X_2650,X_2651,X_2652,X_2653,X_2654,X_2655,X_2656,X_2657,X_2658,X_2659,X_2660,X_2661,X_2662,X_2663,X_2664,X_2665,X_2666,X_2667,X_2668,X_2669,X_2670,X_2671,X_2672,X_2673,X_2674,X_2675,X_2676,X_2677,X_2678,X_2679,X_2680,X_2681,X_2682,X_2683,X_2684,X_2685,X_2686,X_2687,X_2688,X_2689,X_2690,X_2691,X_2692,X_2693,X_2694,X_2695,X_2696,X_2697,X_2698,X_2699,X_2700,X_2701,X_2702,X_2703,X_2704,X_2705,X_2706,X_2707,X_2708,X_2709,X_2710,X_2711,X_2712,X_2713,X_2714,X_2715,X_2716,X_2717,X_2718,X_2719,X_2720,X_2721,X_2722,X_2723,X_2724,X_2725,X_2726,X_2727,X_2728,X_2729,X_2730,X_2731,X_2732,X_2733,X_2734,X_2735,X_2736,X_2737,X_2738,X_2739,X_2740,X_2741,X_2742,X_2743,X_2744,X_2745,X_2746,X_2747,X_2748,X_2749,X_2750,X_2751,X_2752,X_2753,X_2754,X_2755,X_2756,X_2757,X_2758,X_2759,X_2760,X_2761,X_2762,X_2763,X_2764,X_2765,X_2766,X_2767,X_2768,X_2769,X_2770,X_2771,X_2772,X_2773,X_2774,X_2775,X_2776,X_2777,X_2778,X_2779,X_2780,X_2781,X_2782,X_2783,X_2784,X_2785,X_2786,X_2787,X_2788,X_2789,X_2790,X_2791,X_2792,X_2793,X_2794,X_2795,X_2796,X_2797,X_2798,X_2799,X_2800,X_2801,X_2802,X_2803,X_2804,X_2805,X_2806,X_2807,X_2808,X_2809,X_2810,X_2811,X_2812,X_2813,X_2814,X_2815,X_2816,X_2817,X_2818,X_2819,X_2820,X_2821,X_2822,X_2823,X_2824,X_2825,X_2826,X_2827,X_2828,X_2829,X_2830,X_2831,X_2832,X_2833,X_2834,X_2835,X_2836,X_2837,X_2838,X_2839,X_2840,X_2841,X_2842,X_2843,X_2844,X_2845,X_2846,X_2847,X_2848,X_2849,X_2850,X_2851,X_2852,X_2853,X_2854,X_2855,X_2856,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875,Y_Class
0,-0.712645,-1.176847,-0.302095,-1.180272,NaN,-1.182904,-1.179781,NaN,-1.171689,-1.182253,-1.101169,-1.182904,-1.181199,-1.181103,-1.181123,NaN,-0.040893,-1.182904,-1.182904,-1.182904,-1.182904,-1.18111,-1.181023,-1.181005,-1.182904,-1.182452,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182904,-1.182223,-1.176847,-1.182188,-1.181995,-1.182457,-1.182555,-1.182129,-1.182865,-1.182124,-1.181835,-1.182224,-1.182381,-1.181689,-1.182379,-1.182277,-1.181673,-1.182437,-1.182457,-1.182048,-1.182177,-1.182031,-1.182341,-1.181002,-1.182816,-1.17935,-1.182437,-1.182712,-1.182123,-1.18233,-1.182904,-1.182904,-1.182904,NaN,-1.182904,-1.182904,-1.18265,NaN,NaN,NaN,NaN,NaN,NaN,-1.182904,-1.182904,NaN,NaN,NaN,NaN,-1.182883,-1.182883,-1.182883,-1.182904,-1.182595,-1.182904,-0

In [10]:
reg = setup(data = train, target = 'Y_Class', 
            silent=True, session_id = SEED,
            use_gpu=True,
            remove_outliers=False,
            fold=10,
            numeric_features=train_x.columns.to_list(),
            train_size=0.8)

,Description,Value
0,session_id,37
1,Target,Y_Class
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(598, 2878)"
5,Missing Values,True
6,Numeric Features,2877
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


## Classification Model Fit

In [11]:
best5_model = compare_models(fold = 10, n_select = 3)
print('Done.')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7365,0.7732,0.5290,0.7510,0.6962,0.3579,0.4068,0.3800
et,Extra Trees Classifier,0.7363,0.7782,0.5393,0.7378,0.7002,0.3658,0.4021,0.3710
catboost,CatBoost Classifier,0.7322,0.7553,0.5287,0.7251,0.6962,0.3582,0.3981,21.9710
gbc,Gradient Boosting Classifier,0.7258,0.7339,0.5278,0.7073,0.6931,0.3599,0.3872,2.6740
knn,K Neighbors Classifier,0.7238,0.7444,0.5317,0.7079,0.6950,0.3636,0.3852,0.4870
lightgbm,Light Gradient Boosting Machine,0.7154,0.7264,0.5178,0.7058,0.6808,0.3275,0.3570,1.7940
xgboost,Extreme Gradient Boosting,0.7008,0.7137,0.5032,0.6844,0.6674,0.3028,0.3285,1.1240
ada,Ada Boost Classifier,0.6839,0.6586,0.4832,0.6640,0.6539,0.2783,0.2941,0.2610
dummy,Dummy Classifier,0.6674,0.5000,0.3333,0.4455,0.5343,0.0000,0.0000,0.0070
svm,SVM - Linear Kernel,0.6570,0.0000,0.4791,0.6385,0.6348,0.2492,0.2581,0.0320


Done.


In [12]:
tune_models = []
for model in best5_model:
    tune_models.append(tune_model(model,fold=5,n_iter=15))

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7188,0.6952,0.4857,0.7210,0.6660,0.2953,0.3607
1,0.7708,0.8344,0.5451,0.7651,0.7304,0.4356,0.4911
2,0.7188,0.7995,0.5112,0.6884,0.6912,0.3571,0.3728
3,0.7263,0.7393,0.5016,0.7069,0.6876,0.3179,0.3551
4,0.6947,0.7700,0.5562,0.6820,0.6849,0.3361,0.3395
Mean,0.7259,0.7677,0.5200,0.7127,0.6920,0.3484,0.3838
Std,0.0249,0.0480,0.0266,0.0296,0.0211,0.0481,0.0547


In [13]:
# tune model 로 blend 아래 boosting 결과 다음 실행 예정

In [18]:
blended = blend_models(estimator_list = tune_models,fold=5)
# ensemble = ensemble_model (blended, fold = 5, method = 'Boosting')
final_model = finalize_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7083,0.6816,0.4540,0.7697,0.6354,0.2316,0.3321
1,0.7500,0.8188,0.5033,0.7586,0.6953,0.3514,0.4371
2,0.6875,0.7908,0.4642,0.6634,0.6575,0.2777,0.2929
3,0.6947,0.7480,0.4386,0.6735,0.6408,0.2083,0.2490
4,0.7263,0.7735,0.5574,0.7199,0.7064,0.3701,0.3853
Mean,0.7134,0.7625,0.4835,0.7170,0.6671,0.2878,0.3393
Std,0.0226,0.0466,0.0427,0.0431,0.0288,0.0639,0.0664


## Inference

In [19]:
preds = predict_model(final_model, data = test_x)['Label']
print('Done.')

Done.


In [20]:
for i in range(len(train_y.unique())):
    print(len(preds[preds ==i]))

17
273
20


## Submit

In [21]:
current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y_%m_%d_%H_%M")
os.makedirs('./result',exist_ok=True)
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = preds
submit.to_csv(f'./result/baseline_submission_{formatted_datetime}.csv', index=False)